# Data Reading

In [ ]:
# Read in Block data as numpy array
def preprocess_data (data):
    # dtype='U10'
    rows = [list(row) for row in data.split("\n")]
    return np.array(rows, dtype='U10')

In [ ]:
# regex to read numbers

Robot = namedtuple("Robot", ["px", "py","vx", "vy"])

def preprocess_data (data):
    
    def prep_setup(a):
        
        p0, p1, v0, v1 = re.findall(r'-?\d+', a)
        
        return Robot(px=int(p0), py=int(p1), vx=int(v0), vy=int(v1))
    
    rows = [prep_setup(row) for row in data.split("\n")]
    return rows


# Numpy Snippets

In [ ]:
# check if index is in bounds of array
def isValid(shape : tuple, index : tuple):
    return (0 <= index[0] < shape[0] and
            0 <= index[1] < shape[1] and
            0 <= index[2] < shape[2])

In [ ]:
# neightboring cells
directions = [(1,0), (-1,0), (0,1), (0,-1)]
next_nodes = [(row+direction[0], col+direction[1])  for direction in directions 
                                                        if isValid(array.shape, (row+direction[0], col+direction[1])) 
                                                            and isNext(array, (row, col), (row+direction[0], col+direction[1])) ]

In [ ]:
# numpy array with characters to readable str
def display_field(field):
    field_list = field.tolist()
    field_str = "\n".join(["".join(row)   for row in field_list])
    return field_str

# Python Stuff

In [ ]:
Info = namedtuple("Info", ["id", "len", "low", "high"])
info = Info( 1,2,3,4)
info.id

In [ ]:
# Direction methods:
def tuple_add (t1, t2):
        return tuple(map(sum, zip(t1, t2)))

def move_in_dir(position, command):
    if command == '^':
        return tuple_add(position, (-1,0))
    elif command == 'v':
        return tuple_add(position, (1,0))
    elif command == '<':
        return tuple_add(position, (0,-1))
    elif command == '>':
        return tuple_add(position, (0,1))
    else:
        raise ValueError("Unknown Command")
    


In [ ]:
# list of string ints to integers
tuple(map(int, row))
list(map(int, row))

# Directed Graphs and DFS

In [ ]:
zeros = np.argwhere(data == 0)
nines = np.argwhere(data == 9)

print(data)
# index in bounds
def isValid(shape : tuple, index : tuple):
    return (0 <= index[0] < shape[0] and
            0 <= index[1] < shape[1])

# Edge constraint in input 
def isNext(array: np.ndarray, source: tuple, target: tuple):
    if array[source] == array[target] - 1:
        return True
    else:
        return False


def build_dir_graph(array):
    graph = dict()
    for row in range(array.shape[0]):
        for col in range(array.shape[1]):
            # look for edges in 4 directions
            directions = [(1,0), (-1,0), (0,1), (0,-1)]
            next_nodes = [(row+direction[0], col+direction[1])  for direction in directions 
                                                                    if isValid(array.shape, (row+direction[0], col+direction[1])) 
                                                                        and isNext(array, (row, col), (row+direction[0], col+direction[1])) ]
            if next_nodes:
                graph[(row, col)] = next_nodes
    return graph
                
def DFS (graph, array, source):
    leafs = []
    queue = [source]
    
    while queue:
        next = queue.pop(0)

        # print(next)
        # array_2 = array.copy()
        # array_2[next] = -1
        # print(array_2)
        
        # Add more nodes
        if next in graph:
            queue = graph[next] + queue 
        # Leaf constraint 
        elif array[next] == 9:
            leafs.append(next)
            # print("leaf!")
        else:
            pass
            # print("Leaf but not 9")
    return leafs
            


graph = build_dir_graph(data)
print(graph)
total = 0
for start in [tuple(e) for e in zeros.tolist()]:
    leafs = DFS (graph, data, start)
    total += len(leafs)
    print(leafs)
    

return total

In [ ]:
# Dijkstra 

def dijstra (data):
    field = data
    

    def display_field(field):
        field_list = field.tolist()
        field_str = "\n".join(["".join(row)   for row in field_list])
        return field_str

    def isValid(shape : tuple, index : tuple):
        return (0 <= index[0] < shape[0] and
                0 <= index[1] < shape[1])

    def tuple_add (t1, t2):
        return tuple(map(sum, zip(t1, t2)))
    
    def get_neightbors (array, pos):
        directions = [(1,0), (-1,0), (0,1), (0,-1)]
        symbol = ["v", "^", ">", "<"]
        next_nodes = [(tuple_add(pos, direction), symbol) for direction, symbol in zip(directions, symbol) 
                                                                if isValid(array.shape, tuple_add(pos, direction)) 
                                                                   and array[tuple_add(pos, direction)] in ['.', "E"] ]
        return next_nodes
    
    # print(display_field(field))

    # A node contains the position plus the direction which the player faces
    # so we have at most 4 nodes per position on the field
    start = (tuple(np.argwhere(field == 'S')[0]), '>')
    end_nodes = [(tuple(np.argwhere(field == 'E')[0]), dir) for dir in  ["<", ">", "v", "^"]]

    allowed_turns = {"<": ["v", "^"], ">": ["v", "^"], "v":["<", ">"], "^":["<", ">"]}

    # visited nodes -> cannot be in quere anymore
    visited = []
    # nodes to iterate over
    queue = [start]
    # Tracks lowest costs per node
    costs = defaultdict(lambda: float('inf'))
    costs[start] = 0

    while queue:
        # find smallest
        current_node = queue[0]
        # print(current_node)
        lowest_cost = costs[current_node]
        if len(queue) > 1:
            for node in queue[1:]:
                if costs[node] < lowest_cost:
                    current_node = node
                    lowest_cost = costs[node]

        queue.remove(current_node)
        visited.append(current_node)

        nbs = get_neightbors(field, current_node[0])
        # print("Neighbors", nbs)
        for nb in nbs:
            if nb not in visited:
                if current_node[1] == nb[1] and costs[nb] > costs[current_node] +1:
                    costs[nb] = costs[current_node] +1
                    queue.append(nb)
                elif nb[1] in allowed_turns[current_node[1]] and costs[nb] > costs[current_node] +1000:
                    costs[nb] = costs[current_node] +1000+1
                    queue.append(nb)

    end_nodes_costs = [costs[node] for node in end_nodes]

    # display(costs)
    # print(end_nodes)
    return min(end_nodes_costs)



# find all best paths
def dijkstra2 (data, verbose=False):
    field = data
    

    def display_field(field):
        field_list = field.tolist()
        field_str = "\n".join(["".join(row)   for row in field_list])
        return field_str

    def isValid(shape : tuple, index : tuple):
        return (0 <= index[0] < shape[0] and
                0 <= index[1] < shape[1])

    def tuple_add (t1, t2):
        return tuple(map(sum, zip(t1, t2)))
    
    def get_neightbors (array, pos):
        directions = [(1,0), (-1,0), (0,1), (0,-1)]
        symbol = ["v", "^", ">", "<"]
        next_nodes = [(tuple_add(pos, direction), symbol) for direction, symbol in zip(directions, symbol) 
                                                                if isValid(array.shape, tuple_add(pos, direction)) 
                                                                   and array[tuple_add(pos, direction)] in ['.', "E"] ]
        return next_nodes
    
    # print(display_field(field))
    start = (tuple(np.argwhere(field == 'S')[0]), '>')
    end_nodes = [(tuple(np.argwhere(field == 'E')[0]), dir) for dir in  ["<", ">", "v", "^"]]

    allowed_turns = {"<": ["v", "^"], ">": ["v", "^"], "v":["<", ">"], "^":["<", ">"]}

    visited = []
    queue = [start]
    costs = defaultdict(lambda: float('inf'))
    costs[start] = 0
    predecessor = defaultdict(list)

    while queue:
        
        current_node = queue[0]

        # Find lowest cost node in queue (there is a better way probably)
        lowest_cost = costs[current_node]
        if len(queue) > 1:
            for node in queue[1:]:
                if costs[node] < lowest_cost:
                    current_node = node
                    lowest_cost = costs[node]

        queue.remove(current_node)
        visited.append(current_node)

        nbs = get_neightbors(field, current_node[0])
        # if we are on an end node we stop 
        if current_node in end_nodes:
            nbs = []
        
        for nb in nbs:

            # This is problem specific: cost changes depending on turn
            if current_node[1] == nb[1] and costs[nb] >= costs[current_node] +1:

                if costs[nb] == costs[current_node] +1:
                    predecessor[nb].append(current_node)
                else:
                    predecessor[nb] = [current_node]

                # important to check if not already in queue to avoid duplicates
                if nb not in visited and nb not in queue:
                    costs[nb] = costs[current_node] +1
                    queue.append(nb)

            elif nb[1] in allowed_turns[current_node[1]] and costs[nb] >= costs[current_node] +1000+1:
                if costs[nb] == costs[current_node] +1000+1:
                    predecessor[nb].append(current_node)
                else:
                    predecessor[nb] = [current_node]

                # important to check if not already in queue to avoid duplicates
                if nb not in visited and nb not in queue:
                    costs[nb] = costs[current_node] +1000+1
                    queue.append(nb)
            
    
    end_nodes_costs = [costs[node] for node in end_nodes]

    path_field = field.copy()
    pred = [node for node in end_nodes if costs[node] == min(end_nodes_costs)]
    steps = 0
    visited = []
    while pred:
        steps +=1
        current_node = pred.pop()

        # debugging code to display steps
        # path_field[current_node[0]] = 'X'
        # print(display_field(path_field))
        # print(current_node)
        nodes_to_add = [node for node in predecessor[current_node] if node not in visited]
        pred = pred + nodes_to_add

        path_field[current_node[0]] = 'O'
        visited.append(current_node)


    return len(np.argwhere(path_field=='O').tolist())

# dijkstra in extra method for more than one run
def dijkstra3(data):
    def display_field(field):
        field_list = field.tolist()
        field_str = "\n".join(["".join(row)   for row in field_list])
        return field_str
    
    def path_finding(blocks_fallen):
        field_size = data[1]
        blocks = data[0]
        # X,Y coordinate, where 
        #   X is the distance from the left edge of your memory space and 
        #   Y is the distance from the top edge of your memory space

        field = np.full(field_size, '.')
        field[(0,0)] = 'S'
        field[(-1,-1)] = 'E'
        

        bytes_fallen = blocks_fallen
        for i in range(min(len(blocks), bytes_fallen)):
            field[blocks[i][::-1]] = '#'

        # print(display_field(field))

        # Dijkstra
        def isValid(shape : tuple, index : tuple):
            return (0 <= index[0] < shape[0] and
                    0 <= index[1] < shape[1])

        def tuple_add (t1, t2):
            return tuple(map(sum, zip(t1, t2)))
        
        def get_neightbors (array, pos):
            directions = [(1,0), (-1,0), (0,1), (0,-1)]
            symbol = ["v", "^", ">", "<"]
            next_nodes = [tuple_add(pos, direction) for direction in directions
                                                                    if isValid(array.shape, tuple_add(pos, direction)) 
                                                                    and array[tuple_add(pos, direction)] in ['.', 'E'] ]
            return next_nodes

        start = (tuple(np.argwhere(field == 'S')[0]))
        end_nodes = tuple(np.argwhere(field == 'E')[0]) 

        # visited nodes -> cannot be in quere anymore
        visited = []
        # nodes to iterate over
        queue = [start]
        # Tracks lowest costs per node
        costs = defaultdict(lambda: float('inf'))
        costs[start] = 0

        while queue:
            # find smallest
            current_node = queue[0]
            # print(current_node)
            lowest_cost = costs[current_node]
            if len(queue) > 1:
                for node in queue[1:]:
                    if costs[node] < lowest_cost:
                        current_node = node
                        lowest_cost = costs[node]

            queue.remove(current_node)
            visited.append(current_node)

            nbs = get_neightbors(field, current_node)
            # print("Neighbors", nbs)
            for nb in nbs:
                if nb not in visited:
                    if costs[nb] > costs[current_node] +1:
                        costs[nb] = costs[current_node] +1
                        queue.append(nb)


        return costs[end_nodes]
    
    last_solution = data[2]

    upper = len(data[0])
    lower = data[2][0]+1
    memory = {data[2][0]: data[2][1]}

    current  = (upper+lower) // 2
    while lower!=upper:
        steps = path_finding(current)
        memory[current] = steps
        if steps == float("inf"):
            upper = current
        else:
            lower = current +1
        
        current = (upper + lower) // 2

    return data[0][current-1], memory[current-1], memory[current+1]

In [ ]:
# binary search
upper = len(data[0])
lower = data[2][0]+1
memory = {data[2][0]: data[2][1]}

current  = (upper+lower) // 2
while lower!=upper:
    steps = path_finding(current)
    memory[current] = steps
    if steps == float("inf"):
        upper = current
    else:
        lower = current +1
    
    current = (upper + lower) // 2

In [ ]:
# Cached Breadth first search (BFS)

@cache
def build_tree(root):
    
    if root == "":
        return 1
    else:
        next_patterns = next_towels(root, towels)
        ways = 0
        for next_pattern in next_patterns:
            ways += build_tree(next_pattern)
        return ways
        
ways = build_tree(pattern)
total += ways

# MISC

In [ ]:
# For debugging check with some cases

example = r"""#########
#.....#E#
#.....#.#
###.#.#.#
#...#.#.#
#.#.#.#.#
#...#.#.#
#.###.#.#
#S..#...#
#########"""

def test_example_part1(data, answer=None):
    data_prep = preprocess_data(data)
    sol = solution(data_prep)
    if answer:
        print(f"Correct {answer}, got", sol)
    else: 
        print(sol)


test_example_part1(example, answer=3)

In [ ]:
# Error checking of multiple examples

example1 = r"""#########
#.....#E#
#.....#.#
###.#.#.#
#...#.#.#
#.#.#.#.#
#...#.#.#
#.###.#.#
#S..#...#
#########"""

example2 = r"""#########
#....##E#
#.....#.#
###.#.#.#
#...#.#.#
#.#.#.#.#
#...#.#.#
#.###.#.#
#S..#...#
#########"""

example3 = r"""#########
#...#.#E#
#.....#.#
###.#.#.#
#...#.#.#
#.#.#.#.#
#...#.#.#
#.###.#.#
#S..#...#
#########"""

examples = {example1: 7025, example2: 7025, example3: 7025}

def test_part1(examples):
    for idx, (ex, answer) in enumerate(examples.items()):
        data_prep = preprocess_data(ex)
        sol = solution(data_prep)
        if sol != answer:
            print(f"Test example {idx+1} failed.")
        else:
            print(f"Test {idx+1} successful.")
    


test_part1(examples)